<a href="https://colab.research.google.com/github/mohamadrezanumberone/CopyDetection/blob/master/Approximate_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install ptflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ptflops: filename=ptflops-0.6.9-py3-none-any.whl size=11712 sha256=ac5c01bdd3c9d565677f497b1644d8dd3faf87b7ca4d3d00987e350c05b70b21
  Stored in directory: /root/.cache/pip/wheels/c8/71/2f/92426c1ef33fb2e275b533878d8378f91c7f26846d9669019c
Successfully built ptflops


In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import gzip
import shutil
import torchvision
import torch.utils.data as data
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import copy
import time
import os
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torchvision.models as models
import torchsummary
from sklearn.cluster import KMeans
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
%matplotlib inline
from scipy import ndimage
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from collections import OrderedDict 
import math
import torch.quantization.quantize_fx as quantize_fx
from torch.quantization import QuantStub, DeQuantStub
from ptflops import get_model_complexity_info


# Initialization


In [ ]:
num_class = 10
num_epochs = 15
batch_size = 64
feature_extract = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Mounting Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dataset = CIFAR10('./CIFAR10', train= True, transform= transforms.ToTensor(), target_transform = None, download= True)
test_dataset = CIFAR10('./CIFAR10', train= False, transform= transforms.ToTensor(), target_transform = None, download= True)


Files already downloaded and verified
Files already downloaded and verified


# Copying Content

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/SVHN/raw_svhn_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/SVHN/svhn_lbls" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/SVHN/cls_svhn_imgs" "/content/"

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/SVHN/raw_t_svhn_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/SVHN/t_svhn_lbls" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/SVHN/cls_t__svhn_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/CIFAR/raw_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/CIFAR/labels" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/CIFAR/cls_imgs" "/content/"

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/CIFAR/t_raw_imgs" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/CIFAR/t_labels" "/content/"

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/CIFAR/t_clustered" "/content/"

# Load

In [ ]:
img_s = torch.load('raw_svhn_imgs')

In [ ]:
imgs = [torch.tensor(t).squeeze(0).float() for t in img_s]
imgs = torch.stack(imgs, dim=0).float().to(device)
imgs.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([73257, 3, 32, 32])

In [ ]:
t_img_s = torch.load('raw_t_svhn_imgs')

In [ ]:
t_imgs = [torch.tensor(t).squeeze(0).float() for t in t_img_s]
t_imgs = torch.stack(t_imgs, dim=0).float().to(device)
t_imgs.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([26032, 3, 32, 32])

In [ ]:
cls_imgs = torch.load('cls_svhn_imgs')

In [ ]:
clustered_imgs = torch.tensor(cls_imgs).unsqueeze(1).float().to(device)
clustered_imgs.shape


torch.Size([73257, 1, 32, 32])

In [ ]:
t_cls_imgs = torch.load('cls_t__svhn_imgs')

In [ ]:
t_clustered_imgs = torch.tensor(t_cls_imgs).unsqueeze(1).float().to(device)
t_clustered_imgs.shape

torch.Size([26032, 1, 32, 32])

In [ ]:
lbls = torch.load('svhn_lbls')

In [ ]:
lbls = torch.tensor(lbls)
lbls.shape

torch.Size([73257])

In [ ]:
t_lbls = torch.load('t_svhn_lbls')

In [ ]:
t_lbls = torch.tensor(t_lbls)
t_lbls.shape

torch.Size([26032])

In [ ]:
img_s = torch.load('raw_imgs')

In [ ]:
imgs = [torch.tensor(t).squeeze(0).float() for t in img_s]
imgs = torch.stack(imgs, dim=0).float().to(device)
imgs.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([50000, 3, 32, 32])

In [ ]:
t_img_s = torch.load('t_raw_imgs')

In [ ]:
t_imgs = [torch.tensor(t).squeeze(0).float() for t in t_img_s]
t_imgs = torch.stack(t_imgs, dim=0).float().to(device)
t_imgs.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([10000, 3, 32, 32])

In [ ]:
cls_imgs = torch.load('cls_imgs')

In [ ]:
clustered_imgs = torch.tensor(cls_imgs).unsqueeze(1).float().to(device)
clustered_imgs.shape


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


torch.Size([50000, 1, 32, 32])

In [ ]:
t_cls_imgs = torch.load('t_clustered')

In [ ]:
t_clustered_imgs = torch.tensor(t_cls_imgs).unsqueeze(1).float().to(device)
t_clustered_imgs.shape

torch.Size([10000, 1, 32, 32])

In [ ]:
lbls = torch.load('labels')

In [ ]:
lbls = torch.tensor(lbls)
lbls.shape

torch.Size([50000])

In [ ]:
t_lbls = torch.load('t_labels')

In [ ]:
t_lbls = torch.tensor(t_lbls)
t_lbls.shape

torch.Size([10000])

# Custom Dataset

In [ ]:
class CustomDataset(data.Dataset):
    def __init__(self):
        self.raw_images = imgs
        self.clustered_images = clustered_imgs
        self.labels = lbls

    def __getitem__(self, idx):
        image, image_labels, clusteredimage = self.raw_images[idx], self.labels[idx], self.clustered_images[idx]
        return image, image_labels, clusteredimage
    def __len__(self):
        return len(self.raw_images)

In [ ]:
class TestCustomDataset(data.Dataset):
    def __init__(self):
        self.t_raw_images = t_imgs
        self.t_clustered_images = t_clustered_imgs
        self.t_labels = t_lbls

    def __getitem__(self, idx):
        test_image, test_image_labels, testclusteredimage  = self.t_raw_images[idx], self.t_labels[idx], self.t_clustered_images[idx]
        return test_image, test_image_labels, testclusteredimage
    def __len__(self):
        return len(self.t_raw_images)

In [ ]:
# class CustomDataset(data.Dataset):
#     def __init__(self):
#         self.raw_images = imgs
#         self.labels = lbls

#     def __getitem__(self, idx):
#         image, image_labels = self.raw_images[idx], self.labels[idx]
#         return image, image_labels
#     def __len__(self):
#         return len(self.raw_images)

In [ ]:
# class TestCustomDataset(data.Dataset):
#     def __init__(self):
#         self.t_raw_images = t_imgs
#         self.t_labels = t_lbls

#     def __getitem__(self, idx):
#         test_image, test_image_labels  = self.t_raw_images[idx], self.t_labels[idx]
#         return test_image, test_image_labels
#     def __len__(self):
#         return len(self.t_raw_images)

# Dataloader

In [ ]:
train_dataset = CustomDataset()
test_dataset = TestCustomDataset()


In [ ]:
# train, test = torch.utils.data.random_split(train_dataset, [40000, 10000])


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
dataloaders = {'train': train_loader ,'test': test_loader}

In [ ]:
one_train_batch_imgs, one_train_batch_lbls, cls= next(iter(dataloaders['train']))
print(one_train_batch_imgs, '\n', one_train_batch_lbls, '\n', cls)

In [ ]:
one_train_batch_imgs, one_train_batch_lbls= next(iter(dataloaders['train']))
print(one_train_batch_imgs, '\n', one_train_batch_lbls, '\n')

tensor([[[[0.4941, 0.4980, 0.4118,  ..., 0.2314, 0.2784, 0.2824],
          [0.3216, 0.3882, 0.2980,  ..., 0.1373, 0.2275, 0.2902],
          [0.1176, 0.1725, 0.2667,  ..., 0.1216, 0.1804, 0.2902],
          ...,
          [0.1490, 0.1020, 0.0549,  ..., 0.5882, 0.5255, 0.5020],
          [0.2941, 0.1451, 0.0745,  ..., 0.5373, 0.5255, 0.4667],
          [0.3255, 0.2471, 0.1098,  ..., 0.5529, 0.5255, 0.4706]],

         [[0.4980, 0.5059, 0.3961,  ..., 0.2627, 0.3137, 0.3216],
          [0.3255, 0.3843, 0.2706,  ..., 0.1608, 0.2588, 0.3294],
          [0.1176, 0.1569, 0.2235,  ..., 0.1451, 0.2078, 0.3333],
          ...,
          [0.1647, 0.1255, 0.0941,  ..., 0.5255, 0.4510, 0.4196],
          [0.2902, 0.1608, 0.1176,  ..., 0.4667, 0.4549, 0.3961],
          [0.3020, 0.2549, 0.1529,  ..., 0.4784, 0.4627, 0.4157]],

         [[0.5255, 0.5373, 0.4118,  ..., 0.2784, 0.3255, 0.3255],
          [0.3490, 0.4078, 0.2745,  ..., 0.1882, 0.2824, 0.3294],
          [0.1412, 0.1725, 0.2118,  ..., 0

In [ ]:
for batch_number, (a, b, c) in enumerate(dataloaders['train']):
    print(batch_number, a.shape, b.shape, c.shape)

In [ ]:
for batch_number, (a, b) in enumerate(dataloaders['train']):
    print(batch_number, a.shape, b.shape)

0 torch.Size([64, 3, 32, 32]) torch.Size([64])
1 torch.Size([64, 3, 32, 32]) torch.Size([64])
2 torch.Size([64, 3, 32, 32]) torch.Size([64])
3 torch.Size([64, 3, 32, 32]) torch.Size([64])
4 torch.Size([64, 3, 32, 32]) torch.Size([64])
5 torch.Size([64, 3, 32, 32]) torch.Size([64])
6 torch.Size([64, 3, 32, 32]) torch.Size([64])
7 torch.Size([64, 3, 32, 32]) torch.Size([64])
8 torch.Size([64, 3, 32, 32]) torch.Size([64])
9 torch.Size([64, 3, 32, 32]) torch.Size([64])
10 torch.Size([64, 3, 32, 32]) torch.Size([64])
11 torch.Size([64, 3, 32, 32]) torch.Size([64])
12 torch.Size([64, 3, 32, 32]) torch.Size([64])
13 torch.Size([64, 3, 32, 32]) torch.Size([64])
14 torch.Size([64, 3, 32, 32]) torch.Size([64])
15 torch.Size([64, 3, 32, 32]) torch.Size([64])
16 torch.Size([64, 3, 32, 32]) torch.Size([64])
17 torch.Size([64, 3, 32, 32]) torch.Size([64])
18 torch.Size([64, 3, 32, 32]) torch.Size([64])
19 torch.Size([64, 3, 32, 32]) torch.Size([64])
20 torch.Size([64, 3, 32, 32]) torch.Size([64])
21

# Model

In [ ]:
class ConvApprox(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride):

        super(ConvApprox, self).__init__() 
        self.weight = nn.Parameter(torch.Tensor(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = None
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = (kernel_size, kernel_size)
        self.stride = stride
        # unfold
        self.unfold = nn.Unfold(kernel_size=(kernel_size, kernel_size), stride=stride)
        self.reset_parameters()
    
    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, images, clusters):

        # Image input
        img_unfold = self.unfold(images)
        b, c, n = img_unfold.shape

        # Clustered image input
        clusterimg_unfold = self.unfold(clusters)

        img_unfold = img_unfold.view(b, 3, 9, n)
        num_batch, _, num_windows = clusterimg_unfold.size()

        # Mean 
        avg_unfold = torch.mean(img_unfold, dim=2).to(device)

        variance = clusterimg_unfold.var(dim=1, keepdim=True)
        same_ids = variance == 0

        same_data = []
        diff_data = []

        output = torch.zeros((num_batch, 1, self.out_channels, num_windows)).to(device)

        w = self.weight.view(self.out_channels, self.in_channels, -1).to(device)

        for i, s in enumerate(same_ids):
            # Same
            sames = clusterimg_unfold[i, 0, s[0]]
            # Diff
            diffs = clusterimg_unfold[i, :, torch.logical_not(s[0])].t().flatten()
            #
            same_data = torch.matmul(torch.sum(w, dim=-1), avg_unfold[i, :, s[0]]).to(device)
            diff_data = torch.matmul((w).sum(-1) , avg_unfold[i, : ,torch.logical_not(s[0])]).to(device)
            # 
            output[i, 0, :, s[0]] += same_data
            output[i, 0, :, torch.logical_not(s[0])] += diff_data.clone()

        #Fold 
        fold = nn.Fold(output_size=(int(np.sqrt(num_windows)), int(np.sqrt(num_windows))), kernel_size=(1, 1))
        output = fold(output.squeeze(1)).to(device)

        return output

In [ ]:
# class Convnet(nn.Module):
#     def __init__(self):
#         super(Convnet, self).__init__()
#         self.base = models.resnet101(pretrained= True).to(device)
  
     
#     def forward(self, x):
#         y = self.base(x)

#         return y

In [ ]:
class Convnet(nn.Module):
    def __init__(self):
        super(Convnet, self).__init__()
        # self.base = models.vgg16(pretrained= True).to(device)
        base = models.vgg16(pretrained= True).to(device)
        # layers = list(base.children())
        # self.l0 = ConvApprox(3, 64, 3, 1)
        self.l1 = nn.Sequential(*[base.features[i] for i in range(0, 30)])
        self.l2 = nn.Sequential(*[base.avgpool])
        self.l3 = nn.Linear(25088, 64, bias=False).to(device)
        self.l4 = nn.Linear(in_features=64, out_features=10, bias=False).to(device)
        
     
    def forward(self, x):
        y = self.l1(x)
        y = self.l2(y)
        y = y.view(y.size(0), -1)
        y = self.l3(y)
        y = self.l4(y)
 
        return y

In [ ]:
# class Convnet(nn.Module):
#     def __init__(self):
#         super(Convnet, self).__init__()
#         # self.base = models.resnet101(pretrained= True).to(device)
#         base = models.resnet50(pretrained= True).to(device)
#         layers = list(base.children())
#         # self.l0 = ConvApprox(3, 64, 3, 1)
#         self.l1 = nn.Sequential(*layers[0:-1])
#         self.l2 = nn.Linear(2048, 64, bias=False).to(device)
#         self.l3 = nn.Linear(in_features=64, out_features=10, bias=False).to(device)
        
     
#     def forward(self,x):
#         y = self.l1(x)
#         y = y.view(y.size(0), -1)
#         y = self.l2(y)
#         y = self.l3(y)

#         return y

# Load Model

In [ ]:
model = Convnet()
model.eval()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Convnet(
  (l1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [ ]:
model.l1[1]

Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [ ]:
base = models.resnet50(pretrained= True).to(device)
layers = list(base.children())
# layers[0][0]
base

In [ ]:
# base.features[0]

In [ ]:
model = Convnet().to(device)
model
# torchsummary.summary(model, (3, 32, 32),(1, 32, 32))

Convnet(
  (l1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [ ]:
imgs, lbls = next(iter(dataloaders['train']))

In [ ]:
imgs, lbls= next(iter(dataloaders['test']))

In [ ]:
model(imgs.cuda()).to(device)

tensor([[ 1.8430e-01,  1.0322e-01,  1.1524e-01,  7.6123e-02, -4.9111e-02,
          4.7501e-01, -2.0542e-01, -1.6312e-03,  3.1625e-01,  1.5889e-01],
        [-3.9420e-02,  1.3095e-01,  1.1438e-01, -1.3994e-01,  2.7578e-01,
         -1.9204e-01,  1.6807e-01, -7.6512e-02,  1.3753e-01, -2.0022e-01],
        [ 2.4122e-01,  1.2064e-02,  5.5164e-01,  1.6900e-01, -2.7141e-01,
          2.3368e-01,  1.6035e-01, -2.8291e-02,  3.1390e-01, -3.4117e-02],
        [ 1.1104e-02, -1.1950e-01,  8.0807e-02, -1.0937e-02, -5.3778e-02,
          1.0329e-01,  1.6419e-02, -4.2420e-03,  3.4136e-01, -6.2061e-03],
        [ 1.2037e-01, -9.5196e-02,  2.1311e-01,  2.0422e-01,  3.4598e-02,
          1.7099e-01, -1.0383e-01,  7.6558e-02,  9.7153e-02, -9.5707e-02],
        [ 1.3231e-01, -3.1673e-02,  2.2487e-01,  2.4805e-03, -1.2538e-01,
          3.4209e-01, -3.0412e-01,  6.6811e-03,  3.1670e-01, -1.5172e-01],
        [ 8.9200e-03,  3.8776e-02,  1.7576e-01, -3.0295e-03, -3.2036e-04,
          7.5876e-02,  8.9094e-0

In [ ]:
# torchsummary.summary(model,((3, 32, 32), 1, 32, 32) )

In [ ]:
a = next(iter(dataloaders['train']))
a[0].shape
# model(a[0].cuda()).shape

torch.Size([64, 3, 32, 32])

# Prune

In [ ]:
def prune_model_global_unstructured(model, layer_type, proportion):
    module_tups = []
    for module in model.modules():
        if isinstance(module, layer_type):
            module_tups.append((module, 'weight'))

    prune.global_unstructured(
        parameters=module_tups, pruning_method=prune.L1Unstructured,
        amount=proportion
    )
    for module, _ in module_tups:
        prune.remove(module, 'weight')
    return model

In [ ]:
# model = Convnet().to(device)
model = prune_model_global_unstructured(model, nn.Conv2d , 0.3)



In [ ]:
# model = prune_model_global_unstructured(model, nn.BatchNorm2d , 0.8)

In [ ]:
# pruning convapprox
parameters_to_prune = (
    (model.l0, 'weight'),   
)

model_prune = prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.9,
)
prune.remove(model.l0, 'weight')


ConvApprox(
  (unfold): Unfold(kernel_size=(3, 3), dilation=1, padding=0, stride=1)
)

In [ ]:
# pruning fully-connected layer1
parameters_to_prune = (
    (model.l3, 'weight'),   
)

model_prune = prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.3,
)
prune.remove(model.l3, 'weight')


Linear(in_features=25088, out_features=64, bias=False)

In [ ]:
# pruning fully-connected layer2
parameters_to_prune = (
    (model.l4, 'weight'),   
)

model_prune = prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.3,
)
prune.remove(model.l4, 'weight')


Linear(in_features=64, out_features=10, bias=False)

# Save pruned and compress Model

In [ ]:
torch.save(model.state_dict(), "resnet101_CIFAR-80.pt")
!gzip -qf /content/resnet101_CIFAR-80.pt


In [ ]:
save_name = 'VGG16_SVHN_90.pt'
path = F"/content/drive/MyDrive/Colab Notebooks/SVHN/{save_name}" 
torch.save(model.state_dict(), path)

# Sparsity of all Layers

In [ ]:
print(
    "Sparsity in l0.weight: {:.2f}%".format(
        100. * float(torch.sum(model.l0.weight == 0))
        / float(model.l0.weight.nelement())
    )
)
# print(
#     "Sparsity in l1.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.l1.weight == 0))
#         / float(model.l1.weight.nelement())
#     )
# )
print(
    "Sparsity in l2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.l2.weight == 0))
        / float(model.l2.weight.nelement())
    )
)
# print(
#     "Sparsity in l3.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.l3.weight == 0))
#         / float(model.l3.weight.nelement())
#     )
# )
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.l0.weight == 0)
            # + torch.sum(model.l1.weight == 0)
            + torch.sum(model.l2.weight == 0)
            # + torch.sum(model.l3.weight == 0)
        )
        / float(
            model.l0.weight.nelement()
            # + model.l1.weight.nelement()
            + model.l2.weight.nelement()
            # + model.l3.weight.nelement()

        )
    )
)

Sparsity in l0.weight: 89.99%
Sparsity in l2.weight: 90.00%
Global sparsity: 90.00%


In [ ]:
# model = Convnet()

# parameters_to_prune = (
#     (model.l0, 'weight'),
#     (model.l1[1], 'weight'),
#     (model.l1[4], 'weight'),
#     (model.l1[9], 'weight'),
#     (model.l1[11], 'weight'),
#     (model.l1[13], 'weight'),
#     (model.l1[16], 'weight'),
#     (model.l1[18], 'weight'),
#     (model.l1[20], 'weight'),
#     (model.l1[23], 'weight'),
#     (model.l1[25], 'weight'),
#     (model.l1[27], 'weight'),
#     (model.l3, 'weight'),
#     (model.l4, 'weight'),
# )

# prune.global_unstructured(
#     parameters_to_prune,
#     pruning_method=prune.L1Unstructured,
#     amount=0.4,
# )
# prune.remove(model.l0, 'weight')
# prune.remove(model.l1[1], 'weight')
# prune.remove(model.l1[4], 'weight')
# prune.remove(model.l1[9], 'weight')
# prune.remove(model.l1[11], 'weight')
# prune.remove(model.l1[13], 'weight')
# prune.remove(model.l1[16], 'weight')
# prune.remove(model.l1[18], 'weight')
# prune.remove(model.l1[20], 'weight')
# prune.remove(model.l1[23], 'weight')
# prune.remove(model.l1[25], 'weight')
# prune.remove(model.l1[27], 'weight')
# prune.remove(model.l3, 'weight')
# prune.remove(model.l4, 'weight')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Linear(in_features=64, out_features=10, bias=False)

# Config

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Trainer

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):#, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    # if is_inception and phase == 'train':
                    #     # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                    #     outputs, aux_outputs = model(inputs)
                    #     loss1 = criterion(outputs, labels)
                    #     loss2 = criterion(aux_outputs, labels)
                    #     loss = loss1 + 0.4*loss2
                    # else:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history



resnet50-CIFAR-30% pruning (conv2d)

In [ ]:
model_ft, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs)

Epoch 0/14
----------
train Loss: 0.8026 Acc: 0.7271
test Loss: 0.5796 Acc: 0.8054

Epoch 1/14
----------
train Loss: 0.4305 Acc: 0.8547
test Loss: 0.4636 Acc: 0.8461

Epoch 2/14
----------
train Loss: 0.2965 Acc: 0.9023
test Loss: 0.4177 Acc: 0.8613

Epoch 3/14
----------
train Loss: 0.2107 Acc: 0.9295
test Loss: 0.4438 Acc: 0.8585

Epoch 4/14
----------
train Loss: 0.1622 Acc: 0.9455
test Loss: 0.4692 Acc: 0.8669

Epoch 5/14
----------
train Loss: 0.1212 Acc: 0.9595
test Loss: 0.4775 Acc: 0.8682

Epoch 6/14
----------
train Loss: 0.0929 Acc: 0.9681
test Loss: 0.5194 Acc: 0.8674

Epoch 7/14
----------
train Loss: 0.0827 Acc: 0.9721
test Loss: 0.4896 Acc: 0.8763

Epoch 8/14
----------
train Loss: 0.0725 Acc: 0.9759
test Loss: 0.5000 Acc: 0.8731

Epoch 9/14
----------
train Loss: 0.0538 Acc: 0.9818
test Loss: 0.5934 Acc: 0.8621

Epoch 10/14
----------
train Loss: 0.0538 Acc: 0.9828
test Loss: 0.5180 Acc: 0.8764

Epoch 11/14
----------
train Loss: 0.0416 Acc: 0.9866
test Loss: 0.6039 Acc

# Model Parameters


In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
l0.weight 	 torch.Size([64, 3, 3, 3])
l1.1.bias 	 torch.Size([64])
l1.1.weight 	 torch.Size([64, 64, 3, 3])
l1.4.bias 	 torch.Size([128])
l1.4.weight 	 torch.Size([128, 64, 3, 3])
l1.6.bias 	 torch.Size([128])
l1.6.weight 	 torch.Size([128, 128, 3, 3])
l1.9.bias 	 torch.Size([256])
l1.9.weight 	 torch.Size([256, 128, 3, 3])
l1.11.bias 	 torch.Size([256])
l1.11.weight 	 torch.Size([256, 256, 3, 3])
l1.13.bias 	 torch.Size([256])
l1.13.weight 	 torch.Size([256, 256, 3, 3])
l1.16.bias 	 torch.Size([512])
l1.16.weight 	 torch.Size([512, 256, 3, 3])
l1.18.bias 	 torch.Size([512])
l1.18.weight 	 torch.Size([512, 512, 3, 3])
l1.20.bias 	 torch.Size([512])
l1.20.weight 	 torch.Size([512, 512, 3, 3])
l1.23.bias 	 torch.Size([512])
l1.23.weight 	 torch.Size([512, 512, 3, 3])
l1.25.bias 	 torch.Size([512])
l1.25.weight 	 torch.Size([512, 512, 3, 3])
l1.27.bias 	 torch.Size([512])
l1.27.weight 	 torch.Size([512, 512, 3, 3])
l3.weight 	 torch.Size([64, 25088])
l4.weight 	 torch.

# Save Model

In [ ]:
torch.save(model.state_dict(),'densenet_80_after.pt')
!gzip -qf /content/densenet_80_after.pt

# Flops Counter


In [ ]:
with torch.cuda.device(0):
  net = model
  macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True, print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Convnet(
  23.64 M, 100.000% Params, 84.21 MMac, 100.000% MACs, 
  (l1): Sequential(
    23.51 M, 99.443% Params, 84.08 MMac, 99.844% MACs, 
    (0): Conv2d(9.41 k, 0.040% Params, 2.41 MMac, 2.860% MACs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(128, 0.001% Params, 32.77 KMac, 0.039% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(0, 0.000% Params, 16.38 KMac, 0.019% MACs, inplace=True)
    (3): MaxPool2d(0, 0.000% Params, 16.38 KMac, 0.019% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      215.81 k, 0.913% Params, 13.89 MMac, 16.489% MACs, 
      (0): Bottleneck(
        75.01 k, 0.317% Params, 4.83 MMac, 5.730% MACs, 
        (conv1): Conv2d(4.1 k, 0.017% Params, 262.14 KMac, 0.311% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, 0.001% Params, 8.19 KMac, 0.010% MACs, 64, eps=1e-05, momentum=0.1, affine=T

# Model Size

In [ ]:
model = model #models.resnet18()
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 62.259MB


In [ ]:
def print_nonzeros(model):
    nonzero = total = 0
    for name, p in model.named_parameters():
        tensor = p.data.cpu().numpy()
        nz_count = np.count_nonzero(tensor)
        total_params = np.prod(tensor.shape)
        nonzero += nz_count
        total += total_params
        print(f'{name:20} | nonzeros = {nz_count:7} / {total_params:7} ({100 * nz_count / total_params:6.2f}%) | total_pruned = {total_params - nz_count :7} | shape = {tensor.shape}')
    print(f'alive: {nonzero}, pruned : {total - nonzero}, total: {total}, Compression rate : {total/nonzero:10.2f}x  ({100 * (total-nonzero) / total:6.2f}% pruned)')
    return (round((nonzero/total)*100,1))

In [ ]:
print_nonzeros(model)

l1.0.bias            | nonzeros =      64 /      64 (100.00%) | total_pruned =       0 | shape = (64,)
l1.0.weight          | nonzeros =    1676 /    1728 ( 96.99%) | total_pruned =      52 | shape = (64, 3, 3, 3)
l1.2.bias            | nonzeros =      64 /      64 (100.00%) | total_pruned =       0 | shape = (64,)
l1.2.weight          | nonzeros =   32024 /   36864 ( 86.87%) | total_pruned =    4840 | shape = (64, 64, 3, 3)
l1.5.bias            | nonzeros =     128 /     128 (100.00%) | total_pruned =       0 | shape = (128,)
l1.5.weight          | nonzeros =   63608 /   73728 ( 86.27%) | total_pruned =   10120 | shape = (128, 64, 3, 3)
l1.7.bias            | nonzeros =     128 /     128 (100.00%) | total_pruned =       0 | shape = (128,)
l1.7.weight          | nonzeros =  123846 /  147456 ( 83.99%) | total_pruned =   23610 | shape = (128, 128, 3, 3)
l1.10.bias           | nonzeros =     256 /     256 (100.00%) | total_pruned =       0 | shape = (256,)
l1.10.weight         | nonzeros 

70.0

In [ ]:
# get_model_complexity_info(model,)

# Visualization 

In [ ]:
# import matplotlib.pyplot as plt

# plt.plot(loss_values)
# plt.show()